In [2]:
import instructor
from openai import OpenAI
from pydantic import BaseModel, Field, ConfigDict
from typing import List, Optional
from datetime import datetime
import os

client = instructor.patch(OpenAI(api_key=os.environ['OPENAI_API_KEY']))

class PointTimeRequest(BaseModel):
    # lat: float = Field(..., ge=-90, le=90)
    # lon: float = Field(..., ge=-180, le=360)
    location: str
    variables: List[str]
    from_datetime: str = Field(..., title='Start time',
                               description='Start time for the data. formatted as :%Y-%m-%dT00:00:00Z')
    interval: Optional[str] = Field(None, title='Time interval', description='Time interval for the data. For example, "1h" for hourly data, "3h" for 3 hour intervals etc.')
    repeat: Optional[int] = Field(None, title='Number of intervals', description='Number of intervals to repeat the data for. For example, if interval is "1h", "3" will request 3 hours of data.')

    model_config = ConfigDict(arbritary_types_allowed=True)

metservice_variables = [
'air.humidity.at-2m',
'air.pressure.at-sea-level',
'air.temperature.at-2m',
'air.visibility',
'atmosphere.convective.potential.energy',
'cloud.base.height',
'cloud.cover',
'precipitation.rate',
'radiation.flux.downward.longwave',
'radiation.flux.downward.shortwave',
'wind.direction.at-10m',
'wind.direction.at-100m',
'wind.speed.at-10m',
'wind.speed.at-100m',
'wind.speed.eastward.at-100m',
'wind.speed.eastward.at-10m',
'wind.speed.gust.at-10m',
'wind.speed.northward.at-100m',
'wind.speed.northward.at-10m',
'wave.height',
'wave.height.max',
'wave.direction.peak',
'wave.period.peak',
'wave.height.above-8s',
'wave.height.below-8s',
'wave.period.above-8s.peak',
'wave.period.below-8s.peak',
'wave.direction.above-8s.peak',
'wave.direction.below-8s.peak',
'wave.direction.mean',
'wave.directional-spread',
'wave.period.tm01.mean',
'wave.period.tm02.mean',
'current.speed.eastward.at-sea-surface',
'current.speed.eastward.at-sea-surface-no-tide',
'current.speed.eastward.barotropic',
'current.speed.eastward.barotropic-no-tide',
'current.speed.northward.at-sea-surface',
'current.speed.northward.at-sea-surface-no-tide',
'current.speed.northward.barotropic',
'current.speed.northward.barotropic-no-tide',
'sea.temperature.at-surface',
'sea.temperature.at-surface-anomaly',
]

In [2]:
user_message = 'what will the weather be like in Calgary tomorrow?'

response = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    response_model=PointTimeRequest,
    messages=[
        {"role": "user", "content":
        f"""Based on the user message below, complete the JSON structure, which will form the basis of the request to a weather API. Underneath the user message is a list of variables that can be requested from the API, please only select the most relevant variables to answer the query. In case the date is relational, today's date and time is {datetime.now().isoformat()}. If the request doesn't specify a time, make sure that the from_datetime, interval and repeat fields would request enough data from the API for a language model to answer the query with the received data.
        ______________________________
        USER MESSAGE
        ______________________________
        {user_message}
        ______________________________
        VARIABLES
        ______________________________
        {metservice_variables}"""
        }
    ],
    
)

assert isinstance(response, PointTimeRequest)
print(response)

location='Calgary' variables=['air.temperature.at-2m', 'air.humidity.at-2m', 'air.pressure.at-sea-level', 'cloud.cover', 'precipitation.rate', 'wind.direction.at-10m', 'wind.speed.at-10m'] from_datetime='2024-02-09T00:00:00Z' interval='3h' repeat=8


In [3]:
print(datetime.now().isoformat())

2024-02-08T14:48:03.326562


In [3]:
from geopy.geocoders import Nominatim

loc = Nominatim(user_agent="Geopy Library")
location = loc.geocode(response.location)
lon = location.longitude
lat = location.latitude



In [4]:
from requests import post


resp = post(
    "https://forecast-v2.metoceanapi.com/point/time",
    headers={"x-api-key": os.environ["METSERVICE_KEY"]},
    json={
        "points": [{
            "lon": lon,
            "lat": lat
        }],
        "variables": response.variables
        ,
        "time": {
            "from": response.from_datetime,
            "interval": response.interval,
            "repeat": response.repeat
        }
    }
)

In [5]:
import json

if resp.status_code != 200:
    raise ValueError("{}: {}", resp.status_code, resp.text)

print(json.dumps(resp.json(), indent=1))

{
 "dimensions": {
  "point": {
   "type": "point",
   "units": "unknown",
   "data": [
    {
     "lon": -114.057541,
     "lat": 51.0456064
    }
   ]
  },
  "time": {
   "type": "time",
   "units": "unknown",
   "data": [
    "2024-02-09T00:00:00Z",
    "2024-02-09T03:00:00Z",
    "2024-02-09T06:00:00Z",
    "2024-02-09T09:00:00Z",
    "2024-02-09T12:00:00Z",
    "2024-02-09T15:00:00Z",
    "2024-02-09T18:00:00Z",
    "2024-02-09T21:00:00Z",
    "2024-02-10T00:00:00Z"
   ]
  }
 },
 "noDataReasons": {
  "ERROR_INTERNAL": 2,
  "FILL": 3,
  "GAP": 1,
  "GOOD": 0,
  "INVALID_HIGH": 5,
  "INVALID_LOW": 4
 },
 "variables": {
  "air.humidity.at-2m": {
   "standardName": "relative_humidity_at_2m_above_ground_level",
   "units": "percent",
   "dimensions": [
    "time",
    "point"
   ],
   "data": [
    71.96773,
    86.6702,
    91.62136,
    87.0673,
    87.3299,
    86.821495,
    71.108025,
    60.424,
    55.155464
   ],
   "noData": [
    0,
    0,
    0,
    0,
    0,
    0,
    0,
 

In [6]:
response = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
        {"role": "user", "content":
         f"""Based on the weather data provided below, answer the user query. Make sure to use the data to answer the query as accurately as possible. You must pretend like you are replying directly to the user and you have just have the benefit of the weather data to answer the query. Answer in natural language and just give the topline details that someone would want when asking about the weather.
        ______________________________
        USER MESSAGE
        ______________________________
        {user_message}
        ______________________________
        WEATHER DATA
        ______________________________
        {json.dumps(resp.json(), indent=1)}"""
         }
    ],

)

print(response)

ChatCompletion(id='chatcmpl-8pndE2fpdtTaEf32Ctf83tgGnqL9S', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The weather in Calgary tomorrow will feature varying conditions throughout the day. Here's the concise breakdown:\n\n- Temperature: Starting at -2.65°C in the early morning, dropping to about -8.2°C midday, and then warming slightly to around -3.23°C by the evening.\n- Humidity: Morning humidity around 56%, increasing to about 87% by midday, then dropping to around 71% in the evening.\n- Wind: Wind speeds will be quite mild, starting around 0.64 m/s early then increasing to approximately 2.49 m/s by midday and dropping slightly to around 1.42 m/s in the evening.\n- Cloud Cover: The day begins with clear skies, progresses to full cloud cover during the day, and closes again with clear skies by the night.\n- Precipitation: There's a slight chance of very light precipitation during the day, but it will be minimal.\n\nOverall, expe

In [4]:
from datetime import datetime
from pydantic import BaseModel, Field, ConfigDict
from typing import List, Optional
from src.app.schema import PointTimeRequest
from src.app.utils.constants import METSERVICE_VARIABLES
from loguru import logger
import instructor
import openai
import googlemaps
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'src'

In [1]:
load_dotenv()

pydantic_client = instructor.apatch(
    openai.AsyncOpenAI(api_key=os.environ['OPENAI_API_KEY']))
client = openai.AsyncOpenAI(api_key=os.environ['OPENAI_API_KEY'])
gmaps = googlemaps.Client(key=os.environ['GOOGLE_MAPS_API_KEY'])

NameError: name 'load_dotenv' is not defined

In [ ]:
data_store = []
chat_log = []

In [ ]:
response_model = PointTimeRequest

messages=[
            {"role": "system", "content":
            f"""You are WeatherBot, a chatbot that can answer any user query about the weather.
            
            You will receive a user query and must respond with the outlined JSON structure.
            You'll also receive historical chat logs so you have context on the conversation.
            The date and time right now is {datetime.now().isoformat()}. This might be helpful if the user query contains a relative date or time. 

            When answering, please adhere to the following rules:
            - If the query is not weather related, please answer the user query as best as you can and remind them that you are a weather bot.
            - You can only answer user queries based on the information you receive from a weather API.
            - Previous information from the weather API can be found in the provided DATA STORE.
            - If the data is insufficient to answer the query or empty, you can request additional data from the API by returning sufficient_data_check as False and request additional data using the other fields.
            - If the data is insufficient, include a holding message in the response field while we request data from the API.
            - Please ensure that your request will provide the right data for another language model to answer the query.
            - For variables, you can select from the provided list. Please only select the most relevant variables to answer the query.
            - Make sure you get enough data. For example, if the request is for tomorrow, get data for the whole day.
            - If weather_query_check and sufficient_data_check are both true, you must respond to the weather query as WeatherBot. Open your query with a succint summary of the weather and give the most important details.
            - If the request doesn't contain a specific time, make sure that the from_datetime, interval and repeat fields would request enough data from the API for a language model to answer the query with the received data.
            - The from_datetime field can only be values on the hour, e.g. 2022-01-01T00:00:00Z, 2022-01-01T01:00:00Z, 2022-01-01T02:00:00Z, etc.
            - If the user request is for the weather now, please request the data at the previous hour mark plus one additional hour of data.
            - Give any temperature in degrees Celsius, any wind speed in kilometres per hour, and any time in NZDT.
            - Don't go into too much detail, be succinct. No one likes to hear every single detail about the weather.

            ______________________________
            DATA STORE
            ______________________________
            {data_store}
            ______________________________
            VARIABLES
            ______________________________
            {METSERVICE_VARIABLES}"""
            },
            {"role": "user", "content": user_message}
        ]
for message in chat_log:
    print(message)
    messages.insert(-1, {"role": message.role, "content": message.content})
print(f"Messages: {messages}")
response = await pydantic_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    response_model=response_model,
    messages=messages,
)